In [31]:
!apt-get update
!apt-get install -y ffmpeg

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 108 not upgraded.


In [32]:
%pip install openai-whisper pandas datasets[audio]



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
%pip install torchcodec --index-url=https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
%pip install scipy librosa


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import whisper

model = whisper.load_model("turbo")

In [36]:
import librosa

def transcribe_with_whisper(wav):
    # Resample to 16kHz which is Whisper's expected sample rate
    wav_resampled = librosa.resample(wav, orig_sr=audio_samples.sample_rate, target_sr=16000)
    # print(f"\nResampled audio:")
    # print(f"  New shape: {wav_resampled.shape}")
    # print(f"  New duration: {len(wav_resampled) / 16000:.2f} seconds")

    # Transcribe the resampled audio
    result = model.transcribe(
        wav_resampled,
        language="Swedish",
        # verbose=True,
        temperature=0,
        condition_on_previous_text=False,
        # Try without any preprocessing that might affect timing
        no_speech_threshold=0.6,
        logprob_threshold=-1.0
    )

    segments = result["segments"]

    # Check if timestamps make sense now
    audio_duration = len(wav_resampled) / 16000
    segments_cleaned = []

    for i, segment in enumerate(segments):
        text = segment["text"].strip() # type: ignore
        if not text:
            continue
            
        start_time = segment["start"] # type: ignore
        end_time = segment["end"] # type: ignore
        
        # # Debug: Print first few segments to see if timing looks reasonable
        # if i < 5:
        #     print(f"Segment {i}: {start_time:.2f}s - {end_time:.2f}s | {text[:30]}...")
        
        # Only keep segments within audio duration
        if start_time < audio_duration: # type: ignore
            clipped_end = min(end_time, audio_duration)
            segments_cleaned.append({
                "start": start_time,
                "end": clipped_end,
                "text": text,
            })

    return segments_cleaned


In [37]:
import uuid

def write_segments_to_files(segments_cleaned, wav):
    for i, segment in enumerate(segments_cleaned):
        uid = uuid.uuid4()
        start_time = segment["start"]
        end_time = segment["end"]
        text = segment["text"]
        
        start_sample = int(start_time * 24000)
        end_sample = int(end_time * 24000)

        segment_wav = wav[start_sample:end_sample]
        audio_segment_int16 = (segment_wav * 32767).astype(np.int16)

        os.makedirs("audio_segments", exist_ok=True)
        filename = f"segment_{uid}.wav"
        filepath = os.path.join("audio_segments", filename)
        wavfile.write(filepath, 24000, audio_segment_int16)
        
        filename_text = f"segment_{uid}.txt"
        filepath_text = os.path.join("audio_segments", filename_text)
        with open(filepath_text, "w") as f:
            f.write(text)

In [38]:
from scipy.io import wavfile
from datasets import load_dataset
import torch
import os
import numpy as np

dataset = load_dataset("cubbk/audio_swedish_2_dataset_cleaned",split='train', streaming=True, data_dir="8sidor_audios_dataset") 


for i, dataset_item in enumerate(dataset):
    print(f"Processing item {i+1}...")
    audio_decoder = dataset_item[".wav"] # type: ignore
    audio_samples = audio_decoder.get_all_samples()
    wav = audio_samples.data.numpy().squeeze()  
    
    segments = transcribe_with_whisper(wav)
    write_segments_to_files(segments, wav)
    
    if(i == 10):
        break
    
    
    

# filepath = os.path.join("audio_segments", "aaaaaaaaaaa.wav")
# audio_segment_int16 = (wav * 32767).astype(np.int16)
# wavfile.write(filepath, 24000, audio_segment_int16)

Processing item 1...
Processing item 2...
Processing item 3...
Processing item 4...
Processing item 5...
Processing item 6...
Processing item 7...
Processing item 8...
Processing item 9...
Processing item 10...
Processing item 11...


In [39]:
dataset_name = "cubbk/audio_swedish_2_dataset_cleaned"
folder_name = "8sidor_audios_dataset_pairs"
checkpoint_name = "8sidor_audios_dataset_pairs_checkpoint.txt"
segments_until_upload = 140
os.makedirs("archive_to_upload", exist_ok=True)


